<a href="https://colab.research.google.com/github/na2003-gif/Data-training-/blob/main/Compatision_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="border-width:1; border-radius: 15px; border-style: solid; border-color: rgb(10, 10, 10); background-color: #2b6777; color:#ffffff;text-align: center;font: 14pt 'Candara';font-weight:bold;"><h1>Airline passenger satisfaction -Model Comparasion
 </h1></div>

In [1]:
%load_ext cudf.pandas


In [2]:
import kagglehub
teejmahal20_airline_passenger_satisfaction_path = kagglehub.dataset_download('teejmahal20/airline-passenger-satisfaction')
print('Data source import complete.')


Data source import complete.


<div style="border-width:1; border-radius: 15px; border-style: solid; border-color: rgb(10, 10, 10); background-color: #52ab98; text-align: center;font: 14pt 'Candara';font-weight:bold;"><h1>Reading data</h1></div>

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [4]:
data = pd.read_csv("/kaggle/input/airline-passenger-satisfaction/train.csv")

Table dimensions:

In [5]:
data.drop(columns=['Unnamed: 0', 'id'], inplace=True)
data.columns()

Index(['Gender', 'Customer Type', 'Age', 'Type of Travel', 'Class',
       'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes',
       'satisfaction'],
      dtype='object')

In [6]:
data.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 23 columns):
 #   Column                             Non-Null Count   Dtype
---  ------                             --------------   -----
 0   Gender                             103904 non-null  object
 1   Customer Type                      103904 non-null  object
 2   Age                                103904 non-null  int64
 3   Type of Travel                     103904 non-null  object
 4   Class                              103904 non-null  object
 5   Flight Distance                    103904 non-null  int64
 6   Inflight wifi service              103904 non-null  int64
 7   Departure/Arrival time convenient  103904 non-null  int64
 8   Ease of Online booking             103904 non-null  int64
 9   Gate location                      103904 non-null  int64
 10  Food and drink                     103904 non-null  int64
 11  Online boarding                    103904 non-null  int64
 

The first 22 features have been detailed above. The <b>satisfaction</b> feature is the target.

We chaning data to numerical & categorical features.

In [7]:
categorical_indexes = [0, 1, 3, 4] + list(range(6, 20))
data.iloc[:,categorical_indexes] = data.iloc[:,categorical_indexes].astype('category')
data.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 23 columns):
 #   Column                             Non-Null Count   Dtype
---  ------                             --------------   -----
 0   Gender                             103904 non-null  object
 1   Customer Type                      103904 non-null  object
 2   Age                                103904 non-null  int64
 3   Type of Travel                     103904 non-null  object
 4   Class                              103904 non-null  object
 5   Flight Distance                    103904 non-null  int64
 6   Inflight wifi service              103904 non-null  category
 7   Departure/Arrival time convenient  103904 non-null  category
 8   Ease of Online booking             103904 non-null  category
 9   Gate location                      103904 non-null  category
 10  Food and drink                     103904 non-null  category
 11  Online boarding                    103904 no

In [8]:

for col in data.select_dtypes(include='object').columns:
    if data[col].nunique() < 20:
        data[col] = data[col].astype('category')

categorical_columns = [c for c in data.columns if str(data[c].dtype) in ['category']]
numerical_columns = [c for c in data.columns if str(data[c].dtype) in ['int64','float64']]

print("Categorical columns:", categorical_columns)
print("Numerical columns:", numerical_columns)

Categorical columns: ['Gender', 'Customer Type', 'Type of Travel', 'Class', 'Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness', 'satisfaction']
Numerical columns: ['Age', 'Flight Distance', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']


<div style="border-width:1; border-radius: 15px; border-style: solid; border-color: rgb(10, 10, 10); background-color: #52ab98; text-align: center;font: 14pt 'Candara';font-weight:bold;"><h1>Filling in missing values</h1></div>

Let's see how many missing values are in each column of the table:

In [9]:
data.isna().sum()

,0
Gender,0
Customer Type,0
Age,0
Type of Travel,0
Class,0
Flight Distance,0
Inflight wifi service,0
Departure/Arrival time convenient,0
Ease of Online booking,0
Gate location,0


Fill in the missing values with <b>medians</b> in the columns corresponding to quantitative features:

In [10]:
data['Arrival Delay in Minutes'].fillna(data['Arrival Delay in Minutes'].median(axis = 0), inplace = True)

In [11]:
data.isna().sum()

,0
Gender,0
Customer Type,0
Age,0
Type of Travel,0
Class,0
Flight Distance,0
Inflight wifi service,0
Departure/Arrival time convenient,0
Ease of Online booking,0
Gate location,0


In [12]:
data.describe()

,Age,Flight Distance,Departure Delay in Minutes,Arrival Delay in Minutes
count,103904.000000,103904.000000,103904.000000,103904.000000
mean,39.379706,1189.448375,14.815618,15.133392
std,15.114964,997.147281,38.230901,38.649776
min,7.000000,31.000000,0.000000,0.000000
25%,27.000000,414.000000,0.000000,0.000000
50%,40.000000,843.000000,0.000000,0.000000
75%,51.000000,1743.000000,12.000000,13.000000
max,85.000000,4983.000000,1592.000000,1584.000000


This table shows that there are no more missing values.

<div style="border-width:1; border-radius: 15px; border-style: solid; border-color: rgb(10, 10, 10); background-color: #52ab98; text-align: center;font: 14pt 'Candara';font-weight:bold;"><h1>Handling categorical features</h1></div>

We divide the signs into quantitative and categorical:

We divide categorical features into binary and non-binary:

In [13]:

binary_columns = [c for c in categorical_columns if data[c].nunique() == 2 and c != 'satisfaction']
nonbinary_columns = [c for c in categorical_columns if data[c].nunique() > 2]

print("Binary columns:", binary_columns)
print("Non-binary columns:", nonbinary_columns)

Binary columns: ['Gender', 'Customer Type', 'Type of Travel']
Non-binary columns: ['Class', 'Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness']


Let's look at the unique values for each binary feature:

In [14]:
for col in binary_columns:
    print(col, ': ', end = '')
    for uniq in data[col].unique():
        if uniq == data[col].unique()[-1]:
            print(uniq, end = '.')
        else:
            print(uniq, end = ', ')
    print()

Gender : Male, Female.
Customer Type : Loyal Customer, disloyal Customer.
Type of Travel : Personal Travel, Business travel.


Let's do the binarization:

In [15]:
for col in binary_columns:
    data[col] = data[col].astype('object')
    k = 0
    for uniq in data[col].unique():
        data.at[data[col] == uniq, col] = k
        k +=1
for col in binary_columns:
    print(data[col].describe(), end = '\n\n')

count     103904
unique         2
top            1
freq       52727
Name: Gender, dtype: object

count     103904
unique         2
top            0
freq       84923
Name: Customer Type, dtype: object

count     103904
unique         2
top            1
freq       71655
Name: Type of Travel, dtype: object



Now let's look at non-binary categorical features:

In [16]:
data[nonbinary_columns]

,Class,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness
0,Eco Plus,3,4,3,1,5,3,5,5,4,3,4,4,5,5
1,Business,3,2,3,3,1,3,1,1,1,5,3,1,4,1
2,Business,2,2,2,2,5,5,5,5,4,3,4,4,4,5
3,Business,2,5,5,5,2,2,2,2,2,5,3,1,4,2
4,Business,3,3,3,3,4,5,5,3,3,4,4,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103899,Eco,2,1,2,3,2,2,2,2,3,1,4,2,3,2
103900,Business,4,4,4,4,2,4,5,5,5,5,5,5,5,4
103901,Business,1,1,1,3,4,1,5,4,3,2,4,5,5,4
103902,Eco,1,1,1,5,1,1,1,1,4,5,1,5,4,1


The following vectorization method is applicable to non-binary features:

The feature j, which takes s values, will be replaced by s features, which take the values 0 or 1, depending on what the value of the original feature j is.

This vectorization is carried out by the get_dummies method:

In [17]:
data_nonbinary = pd.get_dummies(data[nonbinary_columns])
print(data_nonbinary.columns)

Index(['Class_Business', 'Class_Eco', 'Class_Eco Plus',
       'Inflight wifi service_0', 'Inflight wifi service_1',
       'Inflight wifi service_2', 'Inflight wifi service_3',
       'Inflight wifi service_4', 'Inflight wifi service_5',
       'Departure/Arrival time convenient_0',
       'Departure/Arrival time convenient_1',
       'Departure/Arrival time convenient_2',
       'Departure/Arrival time convenient_3',
       'Departure/Arrival time convenient_4',
       'Departure/Arrival time convenient_5', 'Ease of Online booking_0',
       'Ease of Online booking_1', 'Ease of Online booking_2',
       'Ease of Online booking_3', 'Ease of Online booking_4',
       'Ease of Online booking_5', 'Gate location_0', 'Gate location_1',
       'Gate location_2', 'Gate location_3', 'Gate location_4',
       'Gate location_5', 'Food and drink_0', 'Food and drink_1',
       'Food and drink_2', 'Food and drink_3', 'Food and drink_4',
       'Food and drink_5', 'Online boarding_0', 'Online board

In [18]:
len(data_nonbinary.columns)

86

<div style="border-width:1; border-radius: 15px; border-style: solid; border-color: rgb(10, 10, 10); background-color: #52ab98; text-align: center;font: 14pt 'Candara';font-weight:bold;"><h1>Normalization of quantitative features</h1></div>

We have the following quantitative characteristics:

In [19]:
data_numerical = data[numerical_columns]
data_numerical.describe()

,Age,Flight Distance,Departure Delay in Minutes,Arrival Delay in Minutes
count,103904.000000,103904.000000,103904.000000,103904.000000
mean,39.379706,1189.448375,14.815618,15.133392
std,15.114964,997.147281,38.230901,38.649776
min,7.000000,31.000000,0.000000,0.000000
25%,27.000000,414.000000,0.000000,0.000000
50%,40.000000,843.000000,0.000000,0.000000
75%,51.000000,1743.000000,12.000000,13.000000
max,85.000000,4983.000000,1592.000000,1584.000000


Let's perform a <b>standardization</b> (<em>linear transformation that reduces all values to zero mean and one standard deviation</em>) of all quantitative features:

In [20]:
numerical_columns = data.select_dtypes(include=["number"]).columns.tolist()
data_numerical = data[numerical_columns]
data_numerical = (data_numerical - data_numerical.mean(axis = 0))/data_numerical.std(axis = 0)


In [21]:
data_numerical.describe()

,Age,Flight Distance,Departure Delay in Minutes,Arrival Delay in Minutes
count,103904.000000,103904.000000,103904.000000,103904.000000
mean,-0.000000,0.000000,0.000000,0.000000
std,1.000000,1.000000,1.000000,1.000000
min,-2.142229,-1.161763,-0.387530,-0.391552
25%,-0.819036,-0.777667,-0.387530,-0.391552
50%,0.041038,-0.347440,-0.387530,-0.391552
75%,0.768794,0.555135,-0.073648,-0.055198
max,3.018221,3.804405,41.254178,40.591868


<div style="border-width:1; border-radius: 15px; border-style: solid; border-color: rgb(10, 10, 10); background-color: #52ab98; text-align: center;font: 14pt 'Candara';font-weight:bold;"><h1>Final Table formation</h1></div>

We join all the transformed columns into one table:

In [22]:
target = data['satisfaction']
data = pd.concat([data_numerical, data_nonbinary, data[binary_columns]], axis=1)
print("Data shape after combining features:", data.shape)
data.columns()



Data shape after combining features: (103904, 93)


Index(['Age', 'Flight Distance', 'Departure Delay in Minutes',
       'Arrival Delay in Minutes', 'Class_Business', 'Class_Eco',
       'Class_Eco Plus', 'Inflight wifi service_0', 'Inflight wifi service_1',
       'Inflight wifi service_2', 'Inflight wifi service_3',
       'Inflight wifi service_4', 'Inflight wifi service_5',
       'Departure/Arrival time convenient_0',
       'Departure/Arrival time convenient_1',
       'Departure/Arrival time convenient_2',
       'Departure/Arrival time convenient_3',
       'Departure/Arrival time convenient_4',
       'Departure/Arrival time convenient_5', 'Ease of Online booking_0',
       'Ease of Online booking_1', 'Ease of Online booking_2',
       'Ease of Online booking_3', 'Ease of Online booking_4',
       'Ease of Online booking_5', 'Gate location_0', 'Gate location_1',
       'Gate location_2', 'Gate location_3', 'Gate location_4',
       'Gate location_5', 'Food and drink_0', 'Food and drink_1',
       'Food and drink_2', 'Food and 

Now it looks like this:

In [23]:
data.describe()

,Age,Flight Distance,Departure Delay in Minutes,Arrival Delay in Minutes
count,103904.000000,103904.000000,103904.000000,103904.000000
mean,-0.000000,0.000000,0.000000,0.000000
std,1.000000,1.000000,1.000000,1.000000
min,-2.142229,-1.161763,-0.387530,-0.391552
25%,-0.819036,-0.777667,-0.387530,-0.391552
50%,0.041038,-0.347440,-0.387530,-0.391552
75%,0.768794,0.555135,-0.073648,-0.055198
max,3.018221,3.804405,41.254178,40.591868


In [24]:
corr_mat = data_numerical.corr()
corr_mat

,Age,Flight Distance,Departure Delay in Minutes,Arrival Delay in Minutes
Age,1.000000,0.099461,-0.010152,-0.012105
Flight Distance,0.099461,1.000000,0.002158,-0.002470
Departure Delay in Minutes,-0.010152,0.002158,1.000000,0.960247
Arrival Delay in Minutes,-0.012105,-0.002470,0.960247,1.000000


### Splitting the data on training and test sets

Get <b>X</b> and <b>y</b>:

In [25]:
X = data
y = target
N, d = X.shape
N, d

(103904, 93)

In [26]:
X.columns

Index(['Age', 'Flight Distance', 'Departure Delay in Minutes',
       'Arrival Delay in Minutes', 'Class_Business', 'Class_Eco',
       'Class_Eco Plus', 'Inflight wifi service_0', 'Inflight wifi service_1',
       'Inflight wifi service_2', 'Inflight wifi service_3',
       'Inflight wifi service_4', 'Inflight wifi service_5',
       'Departure/Arrival time convenient_0',
       'Departure/Arrival time convenient_1',
       'Departure/Arrival time convenient_2',
       'Departure/Arrival time convenient_3',
       'Departure/Arrival time convenient_4',
       'Departure/Arrival time convenient_5', 'Ease of Online booking_0',
       'Ease of Online booking_1', 'Ease of Online booking_2',
       'Ease of Online booking_3', 'Ease of Online booking_4',
       'Ease of Online booking_5', 'Gate location_0', 'Gate location_1',
       'Gate location_2', 'Gate location_3', 'Gate location_4',
       'Gate location_5', 'Food and drink_0', 'Food and drink_1',
       'Food and drink_2', 'Food and 

In [27]:
y

,satisfaction
0,neutral or dissatisfied
1,neutral or dissatisfied
2,satisfied
3,neutral or dissatisfied
4,satisfied
...,...
103899,neutral or dissatisfied
103900,satisfied
103901,neutral or dissatisfied
103902,neutral or dissatisfied


Training a neural network takes a lot of time, so let's take not all the data, but only part of it:

In [28]:
N_train = 27000
N_test = 3000

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size = N_train, test_size = N_test,
                                                    stratify = y, random_state = 42)

X_train.shape, X_test.shape

((27000, 93), (3000, 93))

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import pandas as pd

In [46]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5),
    "SVM (RBF Kernel)": SVC(kernel='rbf', C=1.0, gamma='scale', probability=True),
    "MLP (alpha=5.625.623413251903493)": MLPClassifier(
        alpha=5.623413251903491,
        hidden_layer_sizes=(43, 37),
        solver='lbfgs',
        activation='logistic',
        max_iter=1000,
        random_state=42
    )
}

In [47]:

for name, model in models.items():

    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc = accuracy_score(y_test, y_test_pred)

    train_err = 1 - train_acc
    test_err = 1 - test_acc

    results[name] = {
        "Train Accuracy": train_acc,
        "Test Accuracy": test_acc,
        "Training Error": train_err,
        "Test Error": test_err
    }

# Convert to DataFrame for easy viewing
import pandas as pd
results_df = pd.DataFrame(results).T
print(results_df)

                                   Train Accuracy  Test Accuracy  \
Logistic Regression                      0.933296       0.935000   
KNN                                      0.947852       0.926000   
Random Forest                            1.000000       0.959000   
SVM                                      0.964778       0.960333   
MLP (optimized)                          0.979000       0.954333   
K-Nearest Neighbors                      0.947852       0.926000   
SVM (RBF Kernel)                         0.964778       0.960333   
MLP (alpha=5.623)                        0.978333       0.954000   
MLP (alpha=5.625.623413251903493)        0.974963       0.955667   

                                   Training Error  Test Error  
Logistic Regression                      0.066704    0.065000  
KNN                                      0.052148    0.074000  
Random Forest                            0.000000    0.041000  
SVM                                      0.035222    0.039667  